<a href="https://colab.research.google.com/github/nalinis07/APT_Class_Copy_Links/blob/MASTER/AT_Lesson_66_Class_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 66: Variance Inflation Factor

### Teacher-Student Activities

In the previous class, you built a linear regression model again to predict the relative humidity values from the temperature and ozone values. However, you again got `The condition number is large, 3.89e+03. This might indicate that there are strong multicollinearity or other numerical problems.` message after removing the chances of multicollinearity from the model.

In this class, you will learn how to measure the extent of multicollinearity in a multiple linear regression model by calculating the variance inflation factor values for each independent variable. This exercise will enable you to select the features (or independent variables) that predicts the values of the dependent variable best. In machine learning discourse, this exercise is called **feature selection** or **feature elimination**. VIF is one of the ways to select or eliminate features to build a good linear regression model.

Let's quickly run the codes covered in the previous classes and begin this session from **Activity 1: Variance Inflation Factor** section.

---

In [ ]:
# Run the code cell.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Loading the dataset.
csv_file = 'https://s3-student-datasets-bucket.whjr.online/whitehat-ds-datasets/air-quality/AirQualityUCI.csv'
df = pd.read_csv(csv_file, sep=';')

# Dropping the 'Unnamed: 15' & 'Unnamed: 16' columns.
df = df.drop(columns=['Unnamed: 15', 'Unnamed: 16'], axis=1)

# Dropping the null values.
df = df.dropna()

# Creating a Pandas series containing 'datetime' objects.
dt_series = pd.Series(data = [item.split("/")[2] + "-" + item.split("/")[1] + "-" + item.split("/")[0] for item in df['Date']], index=df.index) + ' ' + pd.Series(data=[str(item).replace(".", ":") for item in df['Time']], index=df.index)
dt_series = pd.to_datetime(dt_series)

# Remove the Date & Time columns from the DataFrame and insert the 'dt_series' in it.
df = df.drop(columns=['Date', 'Time'], axis=1)
df.insert(loc=0, column='DateTime', value=dt_series)

# Get the Pandas series containing the year values as integers.
year_series = dt_series.dt.year

# Get the Pandas series containing the month values as integers.
month_series = dt_series.dt.month

# Get the Pandas series containing the day values as integers.
day_series = dt_series.dt.day

# Get the Pandas series containing the days of a week, i.e., Monday, Tuesday, Wednesday etc.
day_name_series = dt_series.dt.day_name()

# Add the 'Year', 'Month', 'Day' and 'Day Name' columns to the DataFrame.
df['Year'] = year_series
df['Month'] = month_series
df['Day'] = day_series
df['Day Name'] = day_name_series

# Sort the DataFrame by the 'DateTime' values in the ascending order. Also, display the first 10 rows of the DataFrame.
df = df.sort_values(by='DateTime')

# Create a function to replace the commas with periods in a Pandas series.
def comma_to_period(series):
    new_series = pd.Series(data=[float(str(item).replace(',', '.')) for item in series], index=df.index)
    return new_series

# Apply the 'comma_to_period()' function on the ''CO(GT)', 'C6H6(GT)', 'T', 'RH' and 'AH' columns.
cols_to_correct = ['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH'] # Create a list of column names.
for col in cols_to_correct: # Iterate through each column
    df[col] = comma_to_period(df[col]) # Replace the original column with the new series.

# Remove all the columns from the 'df' DataFrame containing more than 10% garbage value.
df = df.drop(columns=['NMHC(GT)', 'CO(GT)', 'NOx(GT)', 'NO2(GT)'], axis=1)

# Create a new DataFrame containing records for the years 2004 and 2005.
aq_2004_df = df[df['Year'] == 2004]
aq_2005_df = df[df['Year'] == 2005]

# Replace the -200 value with the median values for each column having indices between 1 and -4 (excluding -4) for the 2004 year DataFrame.
for col in aq_2004_df.columns[1:-4]:
  median = aq_2004_df.loc[aq_2004_df[col] != -200, col].median()
  aq_2004_df[col] = aq_2004_df[col].replace(to_replace=-200, value=median)

# Repeat the same exercise for the 2005 year DataFrame.
for col in aq_2005_df.columns[1:-4]:
  median = aq_2005_df.loc[aq_2005_df[col] != -200, col].median()
  aq_2005_df[col] = aq_2005_df[col].replace(to_replace=-200, value=median)

# Group the DataFrames about the 'Month' column.
group_2004_month = aq_2004_df.groupby(by='Month')
group_2005_month = aq_2005_df.groupby(by='Month')

# Concatenate the two DataFrames for 2004 and 2005 to obtain one DataFrame.
df = pd.concat([aq_2004_df, aq_2005_df])

# Information of the DataFrame.
df.info()

The description for all the columns containing data for air pollutants, temperature, relative humidity and absolute humidity is provided below.


|Columns|Description|
|-|-|
|PT08.S1(CO)|PT08.S1 (tin oxide) hourly averaged sensor response (nominally $\text{CO}$ targeted)|
|C6H6(GT)|True hourly averaged Benzene concentration in $\frac{\mu g}{m^3}$|
|PT08.S2(NMHC)|PT08.S2 (titania) hourly averaged sensor response (nominally $\text{NMHC}$ targeted)|
|PT08.S3(NOx)|PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally $\text{NO}_x$ targeted)|
|PT08.S4(NO2)|PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally $\text{NO}_2$ targeted)|
|PT08.S5(O3) |PT08.S5 (indium oxide) hourly averaged sensor response (nominally $\text{O}_3$ targeted)|
|T|Temperature in Â°C|
|RH|Relative Humidity (%)|
|AH|AH Absolute Humidity|

---

#### Multiple Linear Regression Model Using `sklearn` Module


In [ ]:
# Build a linear regression model using the sklearn module by including all the features listed above.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

features = list(df.columns.values[1:-1])
features.remove('RH')

X = df[features]
y = df['RH']

# Splitting the DataFrame into the train and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42) # Test set will have 33% of the values.

y_train_reshaped = y_train.values.reshape(-1, 1)
y_test_reshaped = y_test.values.reshape(-1, 1)

# Build a linear regression model using the 'sklearn.linear_model' module.
sklearn_lin_reg = LinearRegression()
sklearn_lin_reg.fit(X_train, y_train_reshaped)

# Print the value of the intercept i.e. beta-sub-0.
print("\nConstant".ljust(15, " "), f"{sklearn_lin_reg.intercept_[0]:.6f}") # Soon you will get to know why rounding-off to 6 decimal places.

# Print the names of the features along with the values of their corresponding coefficients.
for item in list(zip(X.columns.values, sklearn_lin_reg.coef_[0])):
  print(f"{item[0]}".ljust(15, " "), f"{item[1]:.6f}") # Soon you will get to know why rounding-off to 6 decimal places.

In [ ]:
# Evaluate the linear regression model using the 'r2_score', 'mean_squared_error' & 'mean_absolute_error' functions of the 'sklearn' module.
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

y_train_pred = sklearn_lin_reg.predict(X_train)
y_test_pred = sklearn_lin_reg.predict(X_test)

print(f"Train Set\n{'-' * 50}")
print(f"R-squared: {r2_score(y_train_reshaped, y_train_pred):.3f}")
print(f"Mean Squared Error: {mean_squared_error(y_train_reshaped, y_train_pred):.3f}")
print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_train_reshaped, y_train_pred)):.3f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_train_reshaped, y_train_pred):.3f}")

print(f"\n\nTest Set\n{'-' * 50}")
print(f"R-squared: {r2_score(y_test_reshaped, y_test_pred):.3f}")
print(f"Mean Squared Error: {mean_squared_error(y_test_reshaped, y_test_pred):.3f}")
print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test_reshaped, y_test_pred)):.3f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test_reshaped, y_test_pred):.3f}")

---

#### The `statsmodels.api` Module

In [ ]:
# Build a linear regression model using the 'statsmodels.api' module.
import statsmodels.api as sm

# Create data frames for the features and target again and also split them into the train and test sets.
X = df[features]
y = df['RH']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42) # Test set will have 33% of the values.

# Add a constant to get an intercept
X_train_sm = sm.add_constant(X_train)

# Fit the regression line using 'OLS'
lr = sm.OLS(y_train, X_train_sm).fit()

# Print the parameters, i.e. the intercept and the slope of the regression line fitted
lr.params

The above values for the constant and the coefficients of all the features are almost the same as the ones obtained through the `sklearn` linear regression model.

In [ ]:
# Performing a summary operation lists out all the different parameters of the regression line fitted
print(lr.summary())

---

#### Multicollinearity

In [ ]:
# Print the summary of the multiple linear regression model built earlier.
print(lr.summary())

Multicollinearity is a situation where the independent variables or features are correlated to each other. Ideally, only the dependent variable (or target) should be correlated with the independent variables and the independent variables should not be correlated with each other at all.


In [ ]:
# Create a heatmap of a correlation DataFrame of the air quality analysis dataset to understand this concept better.
plt.figure(figsize = (12, 6), dpi = 96)
sns.heatmap(df.corr(), annot = True)
plt.show()

From the heatmap, you can see that, the dependent variable `RH` is moderately correlated with `T` and weakly correlated with carbon monoxide (`'PT08.S1(CO)'`), ozone (`'PT08.S5(O3)'`) , absolute humidity (`AH`) and year (`Year`).

Ideally, to build a multiple linear regression model to predict relative humidity, we should have considered carbon monoxide, ozone, absolute humidity and year independent variables only instead of considering all the independent variables. But among these 4 features:

- carbon monoxide and ozone are strongly correlated to each other.

- temperature and absolute humidity are moderately correlated to each other.

- temperature and year are moderately correlated to each other.

- absolute humidity and year are moderately correlated to each other.

The above four cases are examples of multicollinearity wherein the independent variables are correlated to each other.


In [ ]:
# Create a correlation heatmap between 'RH', 'T', 'PT08.S1(CO)', 'PT08.S5(O3)', 'AH', 'Year' variables.
plt.figure(figsize = (6, 4), dpi = 96)
sns.heatmap(df[['RH', 'T', 'PT08.S1(CO)', 'PT08.S5(O3)', 'AH', 'Year']].corr(), annot = True)
plt.show()

This multicollienarity causes redundancy because of which we cannot say for sure which of the independent variables are actually contributing to the prediction of the dependent variable.

In this case, to remove multicollinearity,

- Choose either `T` or `AH` as one of the independent variables. Since the correlation between `RH` and `T` is stronger compared to the correlation between `RH` and `AH`, let's choose `T`.

- Choose either `'PT08.S1(CO)'` or `'PT08.S5(O3)'` as one of the independent variables. Since the correlation between `RH` and `PT08.S5(O3)` is stronger compared to the correlation between `RH` and `'PT08.S1(CO)'`, let's choose `PT08.S5(O3)`.

- Drop `Year` as it is moderately correlated with `'T'`.


In [ ]:
# Create a correlation heatmap between 'RH', 'T', 'PT08.S5(O3)' variables.
plt.figure(figsize = (6, 4), dpi = 96)
sns.heatmap(df[['RH', 'T', 'PT08.S5(O3)']].corr(), annot = True)
plt.show()

Now that we have removed multicollinearity and selected the features that are likely to contribute best to the prediction of relative humidity values, let's build a linear regression model again using the `statsmodels.api` module.

In [ ]:
# Build a linear regression model again with 'T' and 'PT08.S5(O3)' as independent variables to predict 'RH'.
X_train = X_train[['T', 'PT08.S5(O3)']]
X_test = X_test[['T', 'PT08.S5(O3)']]

# Add a constant to get an intercept
X_train_sm1 = sm.add_constant(X_train)

# Fit the regression line using 'OLS'
lr1 = sm.OLS(y_train, X_train_sm1).fit()

# Print the parameters, i.e. the intercept and the slope of the regression line fitted
lr1.params

Let's now print the summary table as well.

In [ ]:
df['PT08.S5(O3)'].std() / np.sqrt(df['PT08.S5(O3)'].mean())
print(df['T'].std() / np.sqrt(df['T'].mean()))

In [ ]:
# Print the summary table to get all the parameters for the features used to build a linear regression model.
print(lr1.summary())

---

#### Activity 1: Variance Inflation Factor Math

Variance Infation Factor (VIF) is a way to detect multicollinearity between independent variables in a dataset. We calculate the VIF values to measure the extent of multicollinearity between the independent variables.  

For $k$ different independent variables, we can calculate $k$ different VIFs (one for each $x_i$ where $i = 1, 2, 3, \dots, k$) in three steps:

**Step one**

First, build a multiple linear regression model wherein $x_i$ is a target variable and it is a function of all the other feature variables as illustrated in the equation below.

$$x_1 = \beta_0^* + \beta_2^* x_2 + \beta_3^* x_3 + \beta_4^* x_4 + \dots + \beta_k^* x_k + \epsilon^*$$

Here,

- $x_1$ is a feature acting as the target (or dependent) variable in above equation

- $x_2, x_3, x_4, \dots , x_k$ are independent variables or features

- $\beta_0^*, \beta_2^*, \beta_3^*, \dots, \beta_k^*$ are the corresponding regression coefficients of the independent variables in the above linear regression equation

- **$\epsilon^*$** is the random error obtained along with the predicted value

**Step two**

Then, calculate the VIF for $x_{i}$ using the following formula:

$$\text{VIF}_{i} = \frac{1}{1-R_{i}^{2}}$$

where $R^2 _i$ is the coefficient of determination of the regression equation in step one, with $x_{1}$ on the left hand side, and all other independent variables on the right hand side.

**Step three**

Analyse the extent of multicollinearity by considering the magnitude of the $\text{VIF}_{i}$. **A rule of thumb is that if $\text{VIF}_{i} > 10$, then multicollinearity is high. In that case, the $x_i$ feature must be dropped to predict the values of the target (or dependent) variable.** A cutoff of 5 is also commonly used.

Let's learn this concept with the help of an example. Let's build a linear regression model to predict relative humidity values from `T` and `PT08.S5(O3)`	values. Let's add one more feature say `'PT08.S1(CO)'` to the prediction mode because ozone and carbon monoxide are highly correlated to each other and relative humidity is correlated to carbon monoxide as well.

Then we will calculate the VIF values for `T, PT08.S5(O3)` and `'PT08.S1(CO)'` independent variables first using the `variance_inflation_factor` function of the `statsmodels.stats.outliers_influence` module and then using $\frac{1}{1 - R^2}$ formula.


In [ ]:
# S1.1: Build a linear regression model again with 'T', 'PT08.S5(O3)' and 'PT08.S1(CO)' as independent variables to predict 'RH'.


In [ ]:
# S1.2: Print the summary table for the above linear regression model.


Now let's calculate the VIF values for `'T', 'PT08.S5(O3)'` and `'PT08.S1(CO)'` independent variables using the `variance_inflation_factor` function of the `statsmodels.stats.outliers_influence` module.

In [ ]:
# S1.3: Calculate the VIF values for 'T', 'PT08.S5(O3)' and 'PT08.S1(CO)' independent variables using the 'variance_inflation_factor' function.


As you can see the VIF values for carbon monoxide and ozone are very high.

Let's learn how to calculate the VIF values using the $\frac{1}{1 - R^2}$ formula. But before that, let's build a linear regression model again taking ozone as the dependent variable and temperature and carbon monoxide as the independent variables. Then calculate the $R^2$ value for this model.

In [ ]:
# S1.4: Build a linear regression model taking 'PT08.S5(O3)' as the target and 'T' and 'PT08.S1(CO)' as the independent variables.


In [ ]:
# S1.5: Calculate the VIF value for ozone where ozone is the dependent variable.


Now, let's repeat the above two exercises to calculate the VIF value for `T` independent variable using the $\frac{1}{1 - R^2}$ variable.

In [ ]:
# S1.6: Build a linear regression model taking 'T' as the target and 'T' and 'PT08.S1(CO)' and 'PT08.S5(O3)' as the independent variables.


In [ ]:
# S1.7: Calculate the VIF value for temperature where temperature is the dependent variable.


---

#### Activity 2: Calculating VIFs for Previously Built Model

Now let's calculate the VIF values for the independent variables in the linear regression model that you built in the previous class.

In [ ]:
# S2.1: Calculate the VIF values for the independent variables in the linear regression model built in the previous class.


Here, the `const` (short for **constant**) is an additional feature that we add before building a linear regression module. It has all the values as $1$. So, `const` is causing multicollinearity.

Ideally, we should drop `const` because the VIF value for it is greater than 10. But we cannot drop it, because using this feature we get the $\beta_0$ value.

So now let's look at the final linear regression model that we have built after selecting the final features that are supposed to predict the relative humidity values.




In [ ]:
# S2.2: Print the summary table for the linear regression model built in the previous class by taking 'T' and 'PT08.S5(O3)' as features.


Hence, to predict relative humidity values, the final linear regression model that you get after going through so much effort is

$$\text{relative humidity} = 64.9564 - 1.1043 \space{} \text{temperature} + 0.0045 \space{} \text{ozone}$$

or

$$\text{RH} = 64.9564 - 1.1043 \space{} \text{T} + 0.0045 \space{} \text{O}_3$$

You can create a function that can predict relative humidity values using the temperature and ozone values.

In [ ]:
# S2.3: Create a function that can predict relative humidity values using the temperature and ozone values.


---

#### Activity 3: Rebuilding Linear Regression Model Using The `sklearn` Module

Let's rebuild the above linear regression model again using the `sklearn` module and then print $R^2$, MSE, RMSE and MAE values.

In [ ]:
# S3.1: Rebuild the above linear regression model again using the sklearn module and then print  𝑅2 , MSE, RMSE and MAE values.


Now let's evaluate the linear regression model using the $R^2$, MSE, RMSE and MAE values.

In [ ]:
# S3.2: Evaluate the linear regression model using the 'r2_score', 'mean_squared_error' & 'mean_absolute_error' functions of the 'sklearn' module.


As you can see, there is hardly any improvement in the performance of the linear regression model after considering ozone as another independent variables to predict the relative humidity values. Hence, we can conclude either of the following from this result:

1. In general, the relative humidity values cannot be predicted accurately using temperature and ozone.

2. In particular, the linear regression model cannot accurately predict the relative humidity values from the ozone and temperature values.

It might happen many a times that a particular problem cannot be solved using a particlar machine learning algorithm. Hence, you might have to use other algorithms to solve that problem. By experience i.e. by solving more and more problems, you will learn to apply the most appropriate machine learning algorithm to solve a problem.

Let's stop here. In the next class, we will solve another problem statement in which linear regression predicts the outcomes more accurately and is best suited for it.

---

### **Project**
You can now attempt the **Capstone Project - Diamond Price Prediction** on your own.

**Capstone Project - Diamond Price Prediction**: https://colab.research.google.com/drive/1YP4OiSvsLVur_2GbEcCnbugflUXrlJP-?usp=sharing

---